# Matek Stat alapozás

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
RANGE = range(10,1000,20)
def calculate_diff_series(lambda_parameter):
    for i in RANGE:
        df=pd.DataFrame()
        df["poi"]=np.random.poisson(lambda_parameter,i)
        df["binom"]=np.random.binomial(10,lambda_parameter/10,i)
        x=df["binom"].value_counts(normalize=True).sort_index()
        y=df["poi"].value_counts(normalize=True).sort_index()
        diff_df = y.to_frame().merge(x.to_frame(), how="left", left_index=True, right_index=True)
        diff_df.fillna(0,inplace=True)
        diff_df["DIFF"] = np.abs(diff_df["poi"] - diff_df["binom"])
        lam_5.append(diff_df["DIFF"].max())
    return  lam_5
lam_1=calculate_diff_series(1)
lam_3=calculate_diff_series(3)
lam_5=calculate_diff_series(5)

konvergencia=pd.DataFrame()
konvergencia["n"]=[i for i in RANGE]
konvergencia["lambda1"]=lam_1
konvergencia["lambda3"]=lam_3
konvergencia["lambda5"]=lam_5

fig  = plt.figure(figsize=(10,10))
ax = plt.axes()

ax.plot(konvergencia["n"], konvergencia["lambda1"], label="lambda=1") 
ax.plot(konvergencia["n"], konvergencia["lambda3"], label="lambda=3") 
ax.plot(konvergencia["n"], konvergencia["lambda5"], label="lambda=5") 

ax.set_xlabel('Eloszlás minta mérete', fontsize=16)
ax.set_ylabel('Sűrűségfüggvények közötti eltérés maximuma', fontsize=16)

ax.legend() # add legend

ax.set_title('Konvergencia különböző lambdákra', fontsize=22)

ValueError: Length of values (200) does not match length of index (50)

In [29]:
lam_1

[0.19999999999999998,
 0.2,
 0.16,
 0.17142857142857143,
 0.10000000000000003,
 0.1545454545454545,
 0.13846153846153847,
 0.07333333333333333,
 0.12941176470588234,
 0.1105263157894737,
 0.057142857142857134,
 0.06521739130434781,
 0.15200000000000002,
 0.1,
 0.08620689655172414,
 0.0838709677419355,
 0.09393939393939393,
 0.08,
 0.07567567567567568,
 0.10256410256410256,
 0.11707317073170734,
 0.13953488372093023,
 0.10444444444444445,
 0.0702127659574468,
 0.1040816326530612,
 0.05882352941176469,
 0.07169811320754718,
 0.05636363636363634,
 0.10526315789473686,
 0.09152542372881359,
 0.08524590163934426,
 0.06349206349206349,
 0.08923076923076925,
 0.09402985074626866,
 0.0811594202898551,
 0.11549295774647889,
 0.08356164383561643,
 0.064,
 0.07402597402597402,
 0.08860759493670886,
 0.07777777777777775,
 0.07710843373493975,
 0.07411764705882354,
 0.067816091954023,
 0.057303370786516844,
 0.1,
 0.0967741935483871,
 0.07999999999999999,
 0.06597938144329898,
 0.05252525252525253,

In [28]:
konvergencia

,n
0,10
1,30
2,50
3,70
4,90
5,110
6,130
7,150
8,170
9,190


In [24]:
lam_5=[]
def calculate_diff_series(lambda_parameter):
    for i in RANGE:
        df=pd.DataFrame()
        df["poi"]=np.random.poisson(lambda_parameter,i)
        df["binom"]=np.random.binomial(10,lambda_parameter*10,i)
        x=df["binom"].value_counts(normalize=True).sort_index()
        y=df["poi"].value_counts(normalize=True).sort_index()
        diff_df = y.to_frame().merge(x.to_frame(), how="left", left_index=True, right_index=True)
        diff_df.fillna(0,inplace=True)
        diff_df["DIFF"] = np.abs(diff_df["poi"] - diff_df["binom"])
        lam_5.append(diff_df["DIFF"].max())
    return  lam_5

In [25]:
lam_5

[0.19999999999999998,
 0.2,
 0.16,
 0.17142857142857143,
 0.10000000000000003,
 0.1545454545454545,
 0.13846153846153847,
 0.07333333333333333,
 0.12941176470588234,
 0.1105263157894737,
 0.057142857142857134,
 0.06521739130434781,
 0.15200000000000002,
 0.1,
 0.08620689655172414,
 0.0838709677419355,
 0.09393939393939393,
 0.08,
 0.07567567567567568,
 0.10256410256410256,
 0.11707317073170734,
 0.13953488372093023,
 0.10444444444444445,
 0.0702127659574468,
 0.1040816326530612,
 0.05882352941176469,
 0.07169811320754718,
 0.05636363636363634,
 0.10526315789473686,
 0.09152542372881359,
 0.08524590163934426,
 0.06349206349206349,
 0.08923076923076925,
 0.09402985074626866,
 0.0811594202898551,
 0.11549295774647889,
 0.08356164383561643,
 0.064,
 0.07402597402597402,
 0.08860759493670886,
 0.07777777777777775,
 0.07710843373493975,
 0.07411764705882354,
 0.067816091954023,
 0.057303370786516844,
 0.1,
 0.0967741935483871,
 0.07999999999999999,
 0.06597938144329898,
 0.05252525252525253]

In [10]:
konvergencia

,n,lambda1,lambda3,lambda5
0,10,0.200000,0.300000,NaN
1,30,0.133333,NaN,0.133333
2,50,0.120000,NaN,NaN
3,70,0.057143,0.128571,0.128571
4,90,0.155556,0.088889,NaN
5,110,0.081818,0.054545,0.081818
6,130,0.107692,0.046154,NaN
7,150,0.033333,0.040000,0.133333
8,170,0.052941,0.076471,0.111765
9,190,0.063158,0.026316,NaN


In [5]:
for i in range(10,1000,20):
    df=pd.DataFrame()
    df["poi"]=np.random.poisson(1,i)
    df["binom"]=np.random.binomial(10,0.1,i)
    x=df["binom"].value_counts(normalize=True).sort_index()
    y=df["poi"].value_counts(normalize=True).sort_index()
    lam_1.append(max(abs(y-x)))
x

0    0.347838
1    0.387182
2    0.194242
3    0.057687
4    0.011131
5    0.001727
6    0.000192
Name: binom, dtype: float64

In [7]:
max(abs(y-x))

0.018373737373737375

In [144]:
import powerlaw as powerlaw
#próbáljuk meg ezt lerajzolni hisztogramon, ahogy az előbb
df=pd.DataFrame()
df["power_2"]=powerlaw.Power_Law(xmin=2, parameters=[2],discrete=True).generate_random(10000)
df["power_2_5"]=powerlaw.Power_Law(xmin=2, parameters=[2.5],discrete=True).generate_random(10000)
df["power_3"]=powerlaw.Power_Law(xmin=2, parameters=[3],discrete=True).generate_random(10000)
df["power_1_5"]=powerlaw.Power_Law(xmin=2, parameters=[1.5],discrete=True).generate_random(10000)

for power in ["power_2", "power_2_5", "power_3", "power_1_5"]:
    counts,bin_edges = np.histogram(df["power_2"], bins=100)

    
    upper=counts*bin_edges[1:]
    lower=counts*bin_edges[0:-1]
    print(f"Eloszláts neve: {power}")
    print(f"Várható érték: {df[power].mean()}")
    print(f"Szórás: {df[power].std()}")
    print(f"Normalizációs konstant: {sum((upper+lower)/2)} \n")

Eloszláts neve: power_2
Várható érték: 14.5193
Szórás: 158.02589422570645
Normalizációs konstant: 624676.7299999999 

Eloszláts neve: power_2_5
Várható érték: 4.9623
Szórás: 18.531584408033986
Normalizációs konstant: 624676.7299999999 

Eloszláts neve: power_3
Várható érték: 3.504
Szórás: 4.558120407159401
Normalizációs konstant: 624676.7299999999 

Eloszláts neve: power_1_5
Várható érték: 1738383.479
Szórás: 168744879.9817331
Normalizációs konstant: 624676.7299999999 

